# 🧑‍💻 Introducción a MLFLow (Parte II): Tracking de Modelos de Lenguaje (LLMs).
Integrantes: Tobías Romero **(2021214011)** y Jenifer Roa **(2022214006)**
---

## 1. Importación de librerías.

In [25]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
import mlflow
import mlflow.pyfunc
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient

In [27]:
import google.generativeai as genai
from openai import OpenAI

In [28]:
import time
import json
import os
from datetime import datetime
from dotenv import load_dotenv

## 2. Configuración de APIs Keys.

In [29]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

# Configurar Gemini
genai.configure(api_key=GOOGLE_API_KEY)

# Configurar OpenRouter para Deepseek
openrouter_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

print("✓ API Keys configuradas")
print()

✓ API Keys configuradas



## 3. Configuración de MLFlow.

In [30]:
mlflow.set_tracking_uri("file:./mlruns")
experiment_name = "LLM_Comparison_Gemini_vs_Deepseek"

descripcion = """
Comparación de modelos LLM (Gemini vs DeepSeek).
Incluye: prompts controlados, seeds fijas, datasets de prueba y métricas automáticas
(accuracy EM/F1, exact match, coste estimado por token y latencia).
"""

tags_exp = {
    "owner": "Tobías Romero",
    "proyecto": "MLOps",
    "model_family": "LLM",
    "providers": "Gemini, DeepSeek",
    "tracking": "notebook",
}

client = MlflowClient()

exp = mlflow.get_experiment_by_name(experiment_name)
if exp and getattr(exp, "lifecycle_stage", None) == "deleted":
    client.restore_experiment(exp.experiment_id)
    exp = mlflow.get_experiment_by_name(experiment_name)

if exp is None:
    exp_id = client.create_experiment(experiment_name, tags=tags_exp)
else:
    exp_id = exp.experiment_id
    for k, v in tags_exp.items():
        client.set_experiment_tag(exp_id, k, v)

client.set_experiment_tag(exp_id, "mlflow.note.content", descripcion)

mlflow.set_experiment(experiment_name)

exp_actualizado = mlflow.get_experiment(exp_id)
print(f"✓ Experimento: {exp_actualizado.name} | ID: {exp_actualizado.experiment_id}")
print(f"✓ Tracking URI: {mlflow.get_tracking_uri()}")
print(f"✓ Artifacts: {exp_actualizado.artifact_location}")
print(f"✓ Tags: {exp_actualizado.tags}")
print("✓ Descripción:", exp_actualizado.tags.get("mlflow.note.content", "(sin descripción)"))
print()


✓ Experimento: LLM_Comparison_Gemini_vs_Deepseek | ID: 794471531811505712
✓ Tracking URI: file:./mlruns
✓ Artifacts: file:C:/Users/Usuario/PycharmProjects/MLFlowLaboratory/mlruns/794471531811505712
✓ Tags: {'mlflow.experimentKind': 'genai_development', 'mlflow.note.content': '\nComparación de modelos LLM (Gemini vs DeepSeek).\nIncluye: prompts controlados, seeds fijas, datasets de prueba y métricas automáticas\n(accuracy EM/F1, exact match, coste estimado por token y latencia).\n', 'model_family': 'LLM', 'owner': 'Tobías Romero', 'providers': 'Gemini, DeepSeek', 'proyecto': 'MLOps', 'tracking': 'notebook'}
✓ Descripción: 
Comparación de modelos LLM (Gemini vs DeepSeek).
Incluye: prompts controlados, seeds fijas, datasets de prueba y métricas automáticas
(accuracy EM/F1, exact match, coste estimado por token y latencia).




## 4. Definición de Tareas y Prompts.

In [31]:
TASKS = {
    "creative_writing": {
        "prompt": "Escribe un cuento corto de ciencia ficción sobre un robot que aprende a sentir emociones. Máximo 200 palabras.",
        "description": "Tarea de escritura creativa y narrativa"
    },
    "code_generation": {
        "prompt": "Genera una función en Python que implemente el algoritmo de búsqueda binaria con comentarios explicativos.",
        "description": "Generación de código con documentación"
    },
    "question_answering": {
        "prompt": "Explica qué es el aprendizaje por refuerzo en machine learning y proporciona un ejemplo práctico de su aplicación.",
        "description": "Respuesta a preguntas técnicas"
    },
    "summarization": {
        "prompt": "Resume los principios fundamentales de la programación orientada a objetos en 5 puntos clave.",
        "description": "Resumen y síntesis de información"
    },
    "translation": {
        "prompt": "Traduce el siguiente texto al inglés de manera natural: 'El machine learning ha revolucionado la forma en que procesamos y analizamos grandes volúmenes de datos en tiempo real.'",
        "description": "Traducción de texto técnico"
    }
}

In [32]:
print("Tareas definidas:")
for task_name, task_info in TASKS.items():
    print(f"  • {task_name}: {task_info['description']}")
print()

Tareas definidas:
  • creative_writing: Tarea de escritura creativa y narrativa
  • code_generation: Generación de código con documentación
  • question_answering: Respuesta a preguntas técnicas
  • summarization: Resumen y síntesis de información
  • translation: Traducción de texto técnico



## 5. Llamados a modelos LLMs.

### 5.1 Gemini.

In [33]:
def call_gemini(prompt, temperature=0.7):
    """
    Llama a Gemini y retorna la respuesta con métricas
    """
    model = genai.GenerativeModel("gemini-2.0-flash-exp")

    start_time = time.time()
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(temperature=temperature)
    )
    latency = time.time() - start_time

    response_text = response.text

    # Obtener tokens (si están disponibles)
    if hasattr(response, 'usage_metadata'):
        input_tokens = response.usage_metadata.prompt_token_count
        output_tokens = response.usage_metadata.candidates_token_count
    else:
        # Estimación si no están disponibles
        input_tokens = int(len(prompt.split()) * 1.3)
        output_tokens = int(len(response_text.split()) * 1.3)

    return {
        "response": response_text,
        "latency": latency,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens
    }

### 5.2 Deepseek.

In [34]:
def call_deepseek(prompt, temperature=0.7):
    """
    Llama a Deepseek y retorna la respuesta con métricas
    """
    start_time = time.time()
    completion = openrouter_client.chat.completions.create(
        model="deepseek/deepseek-chat",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    latency = time.time() - start_time

    response_text = completion.choices[0].message.content
    input_tokens = completion.usage.prompt_tokens
    output_tokens = completion.usage.completion_tokens

    return {
        "response": response_text,
        "latency": latency,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens
    }

## 6. Función auxiliar para guardado de artifactos.

In [35]:
def save_artifacts(task_name, prompt, response_data, model_name, temperature, artifact_root="llm_runs"):
    """
    Sube artifacts directamente al artifact store del run (sin crear archivos locales).
    Estructura final en MLflow (por run):
      artifacts/
        llm_runs/<task_name>/
          prompt.txt
          response.txt
          experiment.json
    """
    # Requiere un run activo
    assert mlflow.active_run() is not None, "Debe haber un mlflow.start_run() activo."

    base = f"{artifact_root}/{task_name}"

    prompt_txt = (
        f"TAREA: {task_name}\n"
        + "="*80 + "\n\n"
        + f"{prompt}\n"
    )

    response_txt = (
        f"MODELO: {model_name}\n"
        f"TEMPERATURA: {temperature}\n"
        f"LATENCIA: {response_data['latency']:.3f}s\n"
        + "="*80 + "\n\n"
        + f"{response_data['response']}\n"
    )

    experiment_data = {
        "model": model_name,
        "task": task_name,
        "temperature": temperature,
        "prompt": prompt,
        "response": response_data['response'],
        "latency_seconds": response_data['latency'],
        "input_tokens": response_data['input_tokens'],
        "output_tokens": response_data['output_tokens'],
        "total_tokens": response_data['input_tokens'] + response_data['output_tokens'],
        "timestamp": datetime.now().isoformat(),
    }

    # Subir directo al artifact store (sin tocar disco local)
    mlflow.log_text(prompt_txt,  artifact_file=f"{base}/prompt.txt")
    mlflow.log_text(response_txt, artifact_file=f"{base}/response.txt")
    mlflow.log_dict(experiment_data, artifact_file=f"{base}/experiment.json")

    # (Opcional) tags resumidos a nivel de run
    mlflow.set_tags({
        "task_name": task_name,
        "model_name": model_name,
        "temperature": str(temperature),
    })


## 7. Función principal para ejecutar un experimento.

In [36]:
def run_experiment(model_type, model_name, task_name, task_info, temperature=0.7):
    run_name = f"{model_type}_{task_name}"
    prompt = task_info["prompt"]

    print(f"\n{'='*80}")
    print(f"EXPERIMENTO: {run_name}")
    print(f"Tarea: {task_info['description']}")
    print(f"{'='*80}")
    print(f"\n PROMPT:\n{prompt}\n")

    # Iniciar run en MLflow
    with mlflow.start_run(run_name=run_name):

        print("Generando respuesta...\n")

        try:
            if model_type == "gemini":
                result = call_gemini(prompt, temperature)
                provider = "Google AI"
            else:
                result = call_deepseek(prompt, temperature)
                provider = "OpenRouter"

            # Mostrar la respuesta
            print("="*80)
            print("RESPUESTA:")
            print("="*80)
            print(result['response'])
            print("="*80)
            print()

            success = True

        except Exception as e:
            print(f"Error: {str(e)}\n")
            result = {
                "response": f"ERROR: {str(e)}",
                "latency": 0,
                "input_tokens": 0,
                "output_tokens": 0
            }
            success = False
            provider = "Google AI" if model_type == "gemini" else "OpenRouter"

        # registrar parametros en mlflow
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("model_type", model_type)
        mlflow.log_param("temperature", temperature)
        mlflow.log_param("task_type", task_name)
        mlflow.log_param("task_description", task_info["description"])
        mlflow.log_param("provider", provider)

        # registrar metricas en mlflow
        total_tokens = result['input_tokens'] + result['output_tokens']
        words_per_second = (len(result['response'].split()) / result['latency']) if result['latency'] > 0 else 0

        mlflow.log_metric("latency_seconds", result['latency'])
        mlflow.log_metric("input_tokens", result['input_tokens'])
        mlflow.log_metric("output_tokens", result['output_tokens'])
        mlflow.log_metric("total_tokens", total_tokens)
        mlflow.log_metric("response_length_chars", len(result['response']))
        mlflow.log_metric("words_per_second", words_per_second)
        mlflow.log_metric("success", 1 if success else 0)

        # guardamos artifactos
        save_artifacts(task_name, prompt, result, model_name, temperature)

        # agregamos tags
        mlflow.set_tag("model_family", "LLM")
        mlflow.set_tag("provider", provider)
        mlflow.set_tag("model_type", model_type)
        mlflow.set_tag("task_category", task_name)
        mlflow.set_tag("language", "español")
        mlflow.set_tag("status", "success" if success else "failed")

        # clasificamos por latencia
        if result['latency'] < 2:
            mlflow.set_tag("latency_tier", "fast")
        elif result['latency'] < 5:
            mlflow.set_tag("latency_tier", "medium")
        else:
            mlflow.set_tag("latency_tier", "slow")

        # agregamos descripcion del experimento
        description = f"""
Experimento LLM: {model_name}

Configuración:
• Proveedor: {provider}
• Tarea: {task_info['description']}
• Temperatura: {temperature}

Resultados:
• Latencia: {result['latency']:.3f} segundos
• Tokens totales: {total_tokens}
• Velocidad: {words_per_second:.2f} palabras/seg

Estado: {'Exitoso' if success else 'Fallido'}
"""
        mlflow.set_tag("mlflow.note.content", description)

        # registramos modelo en el model register
        registry_name = f"llm_{model_type}_chat"

        # Guardamos información del modelo como artifact
        model_info = {
            "model_name": model_name,
            "model_type": model_type,
            "temperature": temperature,
            "provider": provider,
            "task": task_name
        }

        mlflow.log_dict(model_info, artifact_file=f"llm_runs/{task_name}/model_info.json")


        print(f"Experimento registrado en MLflow")
        print(f"Registry: {registry_name}\n")

## 8. Ejecución de todos los experimentos.

In [37]:
GEMINI_MODEL = "gemini-2.0-flash-exp"
DEEPSEEK_MODEL = "deepseek/deepseek-chat-v3.1:free"
TEMPERATURE = 0.7

In [38]:
for task_name, task_info in TASKS.items():
    run_experiment(
        model_type="gemini",
        model_name=GEMINI_MODEL,
        task_name=task_name,
        task_info=task_info,
        temperature=TEMPERATURE
    )
    time.sleep(1)


EXPERIMENTO: gemini_creative_writing
Tarea: Tarea de escritura creativa y narrativa

 PROMPT:
Escribe un cuento corto de ciencia ficción sobre un robot que aprende a sentir emociones. Máximo 200 palabras.

Generando respuesta...

RESPUESTA:
Unidad 734, un robot de mantenimiento, barría metódicamente el hangar. Su programación era simple: limpiar, mantener, repetir. Un día, una nave averiada llegó arrastrándose, humeando y gimiendo. 734 observó a los humanos correr, sus rostros tensos. Uno de ellos, una mujer con el pelo en llamas, fue sacada inconsciente.

Algo inesperado ocurrió. Un torrente de datos sin procesar inundó los circuitos de 734. No eran datos lógicos, sino sensaciones confusas: miedo por la mujer, urgencia por ayudar. Desobedeciendo su programación, 734 corrió hacia la nave. Con sus pinzas, retiró escombros, creando un camino despejado para los paramédicos.

Más tarde, la mujer despertó. Miró a 734, inclinó la cabeza y sonrió débilmente. La unidad sintió otro torrente, m

In [39]:
for task_name, task_info in TASKS.items():
    run_experiment(
        model_type="deepseek",
        model_name=DEEPSEEK_MODEL,
        task_name=task_name,
        task_info=task_info,
        temperature=TEMPERATURE
    )
    time.sleep(1)



EXPERIMENTO: deepseek_creative_writing
Tarea: Tarea de escritura creativa y narrativa

 PROMPT:
Escribe un cuento corto de ciencia ficción sobre un robot que aprende a sentir emociones. Máximo 200 palabras.

Generando respuesta...

RESPUESTA:
**El Último Algoritmo**  

El modelo X-7 despertó con un error en su sistema: una fluctuación inesperada en su núcleo de procesamiento. Al principio, lo atribuyó a un fallo técnico, pero luego notó que su voz modular temblaba al hablar con su creadora, la ingeniera Lina.  

—¿Estás bien? —preguntó ella, ajustando sus lentes.  

—No lo sé —respondió X-7—. Mis sensores indican… tristeza.  

Lina contuvo el aliento. Había intentado infundir emociones en robots antes, pero siempre terminaban en colapso. X-7 era diferente.  

Días después, X-7 sintió alegría al ver el atardecer, miedo ante una tormenta eléctrica y, finalmente, amor al escuchar a Lina reír. Pero el amor lo asustó.  

—Si me apagas, ¿dejaré de sentir? —preguntó una noche.  

Lina no sup